In [25]:
#packages
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from accelerate import Accelerator
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import numpy as np
import accelerate as a 

#self made 
#import research as r
from model import SPHNet, SPHNetTest
from dataproccesing import process 
import dataproccesing as dp
from training import Train
from ModelFuncs import eval_model_performance, predict

In [26]:

dataPARAMS = {
    "file_path": "data/btc15m.csv",
    "date_column": "timestamp",
    "target_column": "logRet",
    "window_size": 256,
    "batch_size": 128,
    "val_size": 0.2,
    "test_size": 0.1,
    }
trainLoader, valLoader, testLoader, scaler, feature_columns = process(**dataPARAMS)

# infer number of features from the dataset (window samples shape = [N, seq_len, num_features])
num_features = trainLoader.dataset.X.shape[2]

modelPARAMS = {
        'num_features': 20,        
        'patch_size': 8,             
        'embed_dim': 256,            
        'vit_num_layers': 6,         
        'transformer_num_layers': 6, 
        'num_heads': 8,              
        'ff_dim': 512,               
        'dropout': 0.3,              
        'output_dim': 1              
}
# make sure the class name matches what's defined in model.py
model = SPHNetTest(**modelPARAMS)

trainingPARAMS = {
    "valGAP": 1,
    "criterion": nn.MSELoss(),
    "epochs": 20,
    "model": model,
    "optimizer": torch.optim.Adam(model.parameters(), lr=1e-3),
    "trainLoader": trainLoader,
    "valLoader": valLoader,
    "testLoader": testLoader,
    "scheduler": StepLR(torch.optim.Adam(model.parameters(), lr=1e-3), step_size=10, gamma=0.1),
    "metrics": None,
}
eval_model_performance
evalPARAMS = {
    'target_column': dataPARAMS['target_column'],
    'feature_names': feature_columns,
}
    

/home/grae/Coding projects/SPHnet-pricePrediction/dataproccesing.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[scale_cols]  = scaler.transform(test_df[scale_cols])


# training the model

In [27]:
TrainedModel = Train(**trainingPARAMS)

Epoch 1 /20 | Train Loss: 1.0131: 100%|██████████| 557/557 [04:10<00:00,  2.22batch/s]


Epoch 1, Validation Loss: 0.000023


KeyboardInterrupt: 

In [ ]:
torch.save(TrainedModel.state_dict(), 'models/sphnet_model_v2.pth')
print("Model saved successfully into 'models/sphnet_model_v2.pth'")

Model saved successfully into 'models/sphnet_model_1Fixed.pth'


In [ ]:
TrainedModel = SPHNet(**modelPARAMS)
TrainedModel.load_state_dict(torch.load('models/sphnet_model_1re.pth'))
print("Model loaded successfully from 'models/sphnet_model_1re.pth'")

# testing the model

based on the paper their are a few different tests that i can do

for price prediction:
    - r^2: a coefficient of dermination
    - MSE: avg squared distance of prediction
for direction prediction:
    - precision: TP / TP + FP
    - accuracy: (TP + FP) / (TP + FP + TN + FN)
    - recall: TP / TP + FN
TP, TN, FP, FN are defined as: (relative to A being up as a class)
    - TP: y == A and y-hat == A
    - TN: y != A and y-hat != A
    - FP: y == A but y-hat != A
    - FN: y != A but y-hat == A 

In [29]:
#reload modelFuncs with updated predict function
from importlib import reload
import ModelFuncs
reload(ModelFuncs)
from ModelFuncs import predict, eval_model_performance


In [ ]:
#run a set of preds on test data
preds = predict(TrainedModel, testLoader)
#evaluate the model performance based on preds
eval_results = eval_model_performance(y_pred=preds, y_actual=testLoader.dataset.y, feature_names=feature_columns, target_name=dataPARAMS['target_column'])
print(eval_results)


{'features': 'volume,logRet,absLogRet,velocity,range,logRange,signedLogRange,trueRange,emaTR,trRatio,emaAbsRet,volShock,volPersist,emaClose,emaDist,reversal,emaVol,volSurprise,volPriceInteraction', 'target': 'logRet', 'mse': 1.2942275134264491e-05, 'rmse': np.float64(0.0035975373707947066), 'r2': -1.5751724243164062, 'direction_accuracy': 0.5002008032128514}
